# Weighted Least Squares

In [ ]:
import gnss_lib_py as glp

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = glp.AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

Solve for the Weighted Least Squares position estimate simply by passing the measurement data.

When obtaining WLS estimates for real measurements, the rotation of the Earth between the signal transmission and reception has to be accounted for.
`solve_wls` accounts for this by default and rotates the given SV positions into the ECEF frame of reference when the signals were received.

In case you assume that the satellite positions are given for the ECEF frame of reference when the signals were received,
this can be accomplished by setting the parameter `tx_time = False` in the function call.

In [ ]:
state_wls = glp.solve_wls(derived_data)
# When assuming that SV positions are given in the ECEF frame when signals are received use
# state_wls = glp.solve_wls(derived_data, use_tx_time=True)

Plot the ECEF x and ECEF y computed position estimate of the receiver

In [ ]:
glp.plot_map(state_wls)

# Extended Kalman Filter

In [ ]:
import gnss_lib_py as glp

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = glp.AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

Solve for the Weighted Least Squares position estimate simply by passing the measurement data.

The implemented EKF also assumes that the satellite positions are given for when the signals were transmitted and that
they must be rotated to account for the change in the ECEF frame of reference.

In this case, to change this default behaviour, pass a dictionary `init_dict` with the key-value pair `"use_tx_time": True`.

To prevent accidental usage of untuned process and measurement noises, `solve_gnss_ekf` requires an initialization \
dictionary containing values for the process noise `"Q"` and the measurement noise `"R"`.

In the naive implementation, the noise matrix is set to the size of the measurement vector using the scalar value `"R"`.

In [ ]:
import numpy as np
init_dict = {"Q": np.eye(7),
             "R" : np.eye(1)}

In [ ]:
state_ekf = glp.solve_gnss_ekf(derived_data, init_dict)

Plot the ECEF x and ECEF y computed position estimate of the receiver

In [ ]:
glp.plot_map(state_ekf)

# Residuals

In [ ]:
import gnss_lib_py as glp

In [ ]:
# load Android Google Challenge data
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/android_2021/Pixel4XL_derived.csv --quiet -O "Pixel4XL_derived.csv"
derived_data = glp.AndroidDerived2021("Pixel4XL_derived.csv", remove_timing_outliers=False)

In [ ]:
galileo_data = derived_data.where("gnss_id","galileo")

Solve for residuals using the estimated state. A new "residuals_m" row is added to derived_data

In [ ]:
glp.solve_residuals(galileo_data, state_wls, inplace=True)

Plot the residuals using the `plot_residuals()` function. The residuals are broken up constellation and signal type and plotted on separate figures.

In [ ]:
figs = glp.plot_metric_by_constellation(galileo_data, "gps_millis", "residuals_m")